### Projekt IUM (zestaw 09, wariant 02)

Autorzy:
- Shpakovich Valeryia 317114
- Wolert Rafał 306020

### Spis treści:
1. Wstęp do projektu
2. Definicja problemu biznesowego 
3. Propozycja rozwiązania problemu biznesowego na podstawie dostępnych danych
    - Wnioski dla danych artystów (artists.jsonl)
    - Wnioski dla danych utworów (tracks.jsonl)
4. Propozycja rozwiązania
5. Kryteria sukcesu
6. Uwagi
7. Analiza danych
   - Załadowanie danych do df
   - Analiza danych 
       * zbiór artystów
       * zbiór utworów 

### Wstęp do projektu

W ramach projektu otrzymano krótki opis biznesowy: “Nie wszyscy nowo dodawani wykonawcy do naszej bazy mają przypisany gatunek muzyczny – musimy jakoś temu zaradzić!”

Dane, które otrzymano od klienta, zawierały:


1. Bazę użytkowników (users.jsonl)

2. Listę wykonawców (artists.jsonl)

3. Listę utworów (tracks.jsonl)

4. Historię sesji użytkowników (sessions.jsonl)

5. Techniczne informacje dotyczące przechowywania plików (track_storage.jsonl)
 
 
 Po krótkiej analizie wstępnej, służącej jako pomoc przy określaniu problemu biznesowego i kryteriów sukcesu, zaobserwowano:

*   Zdecydowano się na wybór danych nr 2 i 3. Pozostałe zbiory nie wnoszą informacji odnośnie wykonawców piosenek oraz samych utworów,
*   W zbiorze listy utworów nie mamy informacji o gatunkach przypisanych do utworów,
*   W zbiorze listy wykonawców mamy wiele gatunków przypisanych do jednego autora,
*   W zbiorze listy wykonawców mamy bardzo dużo podgatunków, które należą do jednego gatunku,
*   W zbiorze listy utworów występuje bezpośredni związek autorów z piosenkami, a zatem gatunków z piosenkami na podstawie tej samej wartości id oraz id_artist odpowiednio w zbiorze artists.jsonl oraz tracks.jsonl

Na podstawie powyższej krótkiej analizy wstępnej określono problem biznesowy.

### Definicja problemu biznesowego

Problemem biznesowym jest brak określonych gatunków muzycznych dla utworów.

Na podstawie powyższych danych, możemy wnioskować o gatunku dla autora muzycznego, tylko na podstawie gatunków muzycznych utworów jego autorstwa.

Poniżej zaprezentowano wizualizację procesu przypisywania gatunków muzycznych do autora na podstawie gatunków muzycznych jego utworów.
![przypisywanie_gatunkow](przypisywanie_gatunkow.png)

### Propozycja rozwiązania problemu biznesowego na podstawie dostępnych danych

#### Wnioski dla danych artystów (artists.jsonl)
Poniższe wnioski zaobserwowano na podstawie danych bez usuwania wartości błędnych.
1. Liczność zbioru artystów: 1667 wierszy na 3 kolumny
2. Istotny błąd w kolumnie 'id'. Mamy 320 wierszy z wartością '-1'. Przez to, nie wiemy jakie piosenki należą do danego artysty w zbiorze danych utworów (tracks.jsonl).
3. Istotny błąd w kolumnie 'genres'. Mamy 328 wartości None - przez to nie można określić, w jakim gatunku tworzy dany artysta.

Poniższe wnioski zaobserwowano na podstawie danych z usuwaniem wartości błędnych (tj. usunięto 328 wierszy z wartościami None w 'genres'

1. Łącznie mamy 5545 gatunków w zbiorze.
2. Podgatunków występujących w liczbie 1, 2, 3, 4 mamy łącznie 567. Pojedynczych podgatunków jest jeszcze więcej i mogą stanowić klasę większościową dla zbioru gatunków.
3. 4/5 najbardziej popularnych unikalnych podgatunków odwołuje się do głównego gatunku "pop". Jest to klasa większościowa, która może negatywnie wpłynać na modelowanie.

Wniosek: Potrzeba dla artystów, w których nie występują błędy zarówno w kolumnie 'id' jak i 'genres'. Wtedy, możliwe, że niezbalansowanie danych zaobserwowanie w zbiorze przy usuniętych wartościach None będzie akceptowalne dla zadań modelowania.

#### Wnioski dla danych utworów (tracks.jsonl)
Poniższe wnioski zaobserwowano na podstawie danych bez usuwania wartości błędnych.

1. Liczność zbioru to 22412 wiersze na 7 kolumn.
2. Wartości typu None w kolumnach: 'id', 'name', 'popularity', 'id_artist'. Istotne błędy w danych to kolumny: 'name', 'popularity' oraz 'id_artist'. Błędy typu 'id' jesteśmy w stanie uzupełnić generując id, przypisując kolejne cyfry do piosenek - nie jest to rażący błąd danych. 

Brak nazw piosenek w 'name' nie jest możliwy do uzupełnienia przez nasz zespół analityków, nie jesteśmy w stanie wtedy wykryć np.: duplikatów piosenek po nazwie, które mają podobne cechy piosenek takie jak: energy, danceability, key itd.

Brak wskaźnika popularności w 'popularity' - może nie być to kluczowa cecha w przypadku zadań modelowania, jednak ciężko jest uzupełnić tak występujący brak danych reprezentatywnie. Możemy wypełnić braki np.: często występującą wartością, ale ilość braków dla tej kolumny (4436 braków) jest duża.

Celem dalszej analizy, usunięto braki None w trzech kolumnach pozostałych kolumnach (łącznie 13 446 rekordów). Braki w kolumnie 'id' czyli 4480, uzupełniono kolejnymi cyframi od 0.

Poniższe wnioski zaobserwowano na podstawie danych z usuwaniem wartości błędnych (tj. usunięto 13 446 wierszy z wartościami None w trzech kolumnach.

1. Z pozostałych danych, 11 451 rekordów, sprawdziliśmy ile artystów ze zbioru artists.jsonl występuje w zbiorze piosenek jako autorzy. Jest to 9092 artystów, którzy mają odwzorowanie w zbiorze artists.jsonl, a co za tym idzie, mają przypisane gatunki, w których tworzą. 2359 artystów z tracks.jsonl nie znajduje się w zbiorze artists.jsonl

Wniosek: Potrzebny zbiór danych, który nie zawiera braków w 'name', 'popularity', 'id_artist'. Spora część danych, 9092 artystów ze zbioru piosenek ma swoje odwzorowanie w zbiorze artystów, co powinno wystarczyć w celach modelowania.


### Propozycja rozwiązania

1. Proponujemy bardziej uogólniony podział na gatunki. Bez uogólnionego podziału, istnieje bardzo dużo podgatunków w małej liczności w zbiorze. Oznacza to, że model nie będzie w stanie generalizować przypadków gatunków, pojedyncze podgatunki występujące w zbiorze wpływają negatywnie na jego działanie.
W tym przypadku użyjemy **metod asocjacyjnych**, które są w stanie skutecznie powiązać podobne elementy ze sobą i w ten sposób zredukować liczność zbioru.
2.  Mając uogólnione gatunki, jesteśmy w stanie przejść do pierwszego z zadań modelowania:
  - na zbiorze wszystkich piosenek stosujemy **clustering**, celem uzyskania grup, które posiadają cechy charakterystyczne dla każdego gatunku i są związane z rytmicznością, tanecznością, głośnością itd. danego utworu
  - wynikiem clusteringu są grupy, z których w każdej znajdują się piosenki z przypisanymi autorami. Artyści mają przypisane gatunki w których tworzą.
  - na podstawie gatunków, w których artyści tworzą, z pomocą heurystyk, jaki gatunek występuje w grupie najczęściej, określany będzie gatunek całej grupy.

  Dzięki temu, otrzymamy dodatkową informację o gatunku danego utworu.
  Powyższy opis zaprezentowano na poniższym diagramie.

  ![pierwszy_model](pierwszy_model.png)

Dla grupowania danych prawdopodobnie zostanie wykorzystana metoda k-średnich, na podstawie danej mapy: https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html

3. Drugie zadanie modelowania będzie polegało na:
  - pracy ze zbiorem początkowym, trenującym, ale wzbogaconym o informację o gatunku dla danego utworu,
  - na podstawie tych danych, tworzymy model, który określa gatunek piosenki (zadanie klasyfikacji)

Końcowy model utworzony w ramach drugiego zadania modelowania będzie wykorzystywany "na produkcji". Algorytm użyty do zadania klasyfikacji zostanie określony po otrzymaniu wyników z modelu pierwszego.

Przedstawiona architektura wymaga aktualizacji danych treningowych do modelu 1 i 2, co pewien określony czas.

### Kryteria sukcesu

**Biznesowe:**  

Nowododanym wykonawcom skutecznie przypisywane są gatunki muzyczne. 

**Analityczne:**  

Ostateczne kryterium sukcesu: jak dobrze dobierane są gatunki do konkretnego wykonawcy? Na przykład, na ile prawdopodobieństwo tego, że nasze rozwiązanie przypisze do wykonawcy odpowiedni gatunek, jest lepsze od losowego przypisania. 

Podczas pracy do określenia jakości implementacji i dobranych parametrów zostaną dobrane specyficzne dla problemów clusteringu (grupowania) i klasyfikacji metryki takie jak:
1. dla clusteringu:
  - Silhouette Coefficient
  - Dunn's Index
2. dla klasyfikacji:
  - confusion matrix
  - ROC curve
  - PR curve
  - Log Loss

### Uwagi:
W dalszej części notatnika przeprowadzono analizę danych, na podstawie której oceniono czy dane są wystarczające, czy w danych czegoś brakuje. Należy nadmienić, że finalnie, tak jak powyżej, stwierdzono, że należy przesłać dane bez błędów, które posłużą do dalszej analizy. Z tego względu, w analizie danych nie skupiowano się na przydatności otrzymanych cech, np.: w przypadku zbioru tracks.jsonl. Zostanie to wykonane, po otrzymaniu zbioru niezawierającego błędów.

Poniższa analiza składa się z sekcji:
1. Załadowanie danych do df
2. Analiza danych 
    * zbiór artystów
    * zbiór utworów      

### 1. Załadowanie danych do df

In [314]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


In [315]:
def load_lines(filename):
    with open(filename) as f:
        lines = f.read().splitlines()
    return lines

In [316]:
artists_lines = load_lines("artists.jsonl")
tracks_lines = load_lines("tracks.jsonl")

In [317]:
artists_lines[:10]

['{"id": "7jVv8c5Fj3E9VhNjxT4snq", "name": "Lil Nas X", "genres": ["country rap", "lgbtq+ hip hop", "pop rap", "queer country"]}',
 '{"id": "0xRXCcSX89eobfrshSVdyu", "name": "MEDUZA", "genres": ["dance pop", "edm", "pop dance", "pop house", "tropical house"]}',
 '{"id": -1, "name": "TheFatRat", "genres": ["speedrun"]}',
 '{"id": "4f7KfxeHq9BiylGmyXepGt", "name": "Tanishk Bagchi", "genres": ["desi pop", "filmi", "modern bollywood"]}',
 '{"id": -1, "name": "Jawsh 685", "genres": ["nz pop"]}',
 '{"id": "523y9KSneKh6APd1hKxLuF", "name": "Master KG", "genres": ["south african house"]}',
 '{"id": "25UNJbwGZSQKvz5cPLWlv3", "name": "Los Dos Carnales", "genres": null}',
 '{"id": "6vTqEFbTtTRJsuIpzZgjxi", "name": "Grupo Menos \\u00c9 Mais", "genres": ["pagode", "pagode novo"]}',
 '{"id": "44mEtidu0VdRkIqO4IbkNa", "name": "Marca MP", "genres": ["corrido", "corridos tumbados", "nuevo regional mexicano", "regional mexican", "sierreno"]}',
 '{"id": -1, "name": "Guilherme & Benuto", "genres": ["serta

In [318]:
tracks_lines[:10]

['{"id": "0RNxWy0PC3AyH4ThH3aGK6", "name": null, "popularity": null, "duration_ms": 201467, "explicit": 0, "id_artist": "19eLuQmk9aCobbVDHc6eek", "release_date": "1929", "danceability": 0.673, "energy": 0.377, "key": 0, "loudness": -14.141, "speechiness": 0.0697, "acousticness": 0.586, "instrumentalness": 0.0, "liveness": 0.332, "valence": 0.713, "tempo": 88.973}',
 '{"id": "2W889aLIKxULEefrleFBFI", "name": "Someone to Watch Over Me", "popularity": 54, "duration_ms": 198000, "explicit": 0, "id_artist": "1Mxqyy3pSjf8kZZL4QVxS0", "release_date": "1943", "danceability": 0.204, "energy": 0.151, "key": 2, "loudness": -17.842, "speechiness": 0.0418, "acousticness": 0.947, "instrumentalness": 9.15e-06, "liveness": 0.321, "valence": 0.134, "tempo": 91.783}',
 '{"id": "4Pnzw1nLOpDNV6MKI5ueIR", "name": null, "popularity": 55, "duration_ms": 199000, "explicit": 0, "id_artist": "1Mxqyy3pSjf8kZZL4QVxS0", "release_date": "1944", "danceability": 0.295, "energy": 0.0826, "key": 1, "loudness": -19.569,

In [319]:
artists_df = pd.DataFrame(artists_lines)
tracks_df = pd.DataFrame(tracks_lines)

In [320]:
artists_df

,0
0,"{""id"": ""7jVv8c5Fj3E9VhNjxT4snq"", ""name"": ""Lil ..."
1,"{""id"": ""0xRXCcSX89eobfrshSVdyu"", ""name"": ""MEDU..."
2,"{""id"": -1, ""name"": ""TheFatRat"", ""genres"": [""sp..."
3,"{""id"": ""4f7KfxeHq9BiylGmyXepGt"", ""name"": ""Tani..."
4,"{""id"": -1, ""name"": ""Jawsh 685"", ""genres"": [""nz..."
...,...
1662,"{""id"": ""6AMd49uBDJfhf30Ak2QR5s"", ""name"": ""Coi ..."
1663,"{""id"": ""6qgnBH6iDM91ipVXv28OMu"", ""name"": ""KAYT..."
1664,"{""id"": ""5q8HGNo0BjLWaTAhRtbwxa"", ""name"": ""Bran..."
1665,"{""id"": -1, ""name"": ""Bethel Music"", ""genres"": [..."


In [321]:
tracks_df

,0
0,"{""id"": ""0RNxWy0PC3AyH4ThH3aGK6"", ""name"": null,..."
1,"{""id"": ""2W889aLIKxULEefrleFBFI"", ""name"": ""Some..."
2,"{""id"": ""4Pnzw1nLOpDNV6MKI5ueIR"", ""name"": null,..."
3,"{""id"": ""7GLmfKOe5BfOXk7334DoKt"", ""name"": ""Satu..."
4,"{""id"": ""6kD1SNGPkfX9LwaGd1FG92"", ""name"": ""Put ..."
...,...
22407,"{""id"": ""0LcNMuOiULmxJK3bdHTfDF"", ""name"": ""Miss..."
22408,"{""id"": ""1uviKYHZuM4uINK33F7sCt"", ""name"": ""Fix ..."
22409,"{""id"": null, ""name"": ""Lotus Inn"", ""popularity""..."
22410,"{""id"": ""27kcZEJvhkb1rzZS9gCpdA"", ""name"": null,..."


In [322]:
artists_df.columns = ['json line']
tracks_df.columns = ['json line']

# utworzenie słownika na każdym elemencie kolumny json line
loaded_json_artists = artists_df['json line'].apply(json.loads)
loaded_json_tracks = tracks_df['json line'].apply(json.loads)


In [323]:
artists_df = pd.json_normalize(loaded_json_artists)
tracks_df = pd.json_normalize(loaded_json_tracks)

In [324]:
artists_df

,id,name,genres
0,7jVv8c5Fj3E9VhNjxT4snq,Lil Nas X,"[country rap, lgbtq+ hip hop, pop rap, queer c..."
1,0xRXCcSX89eobfrshSVdyu,MEDUZA,"[dance pop, edm, pop dance, pop house, tropica..."
2,-1,TheFatRat,[speedrun]
3,4f7KfxeHq9BiylGmyXepGt,Tanishk Bagchi,"[desi pop, filmi, modern bollywood]"
4,-1,Jawsh 685,[nz pop]
...,...,...,...
1662,6AMd49uBDJfhf30Ak2QR5s,Coi Leray,"[pop r&b, r&b, trap queen, viral rap]"
1663,6qgnBH6iDM91ipVXv28OMu,KAYTRANADA,"[escape room, indie soul, lgbtq+ hip hop]"
1664,5q8HGNo0BjLWaTAhRtbwxa,Brantley Gilbert,"[contemporary country, country, country road, ..."
1665,-1,Bethel Music,"[ambient worship, ccm, christian music, deep c..."


In [325]:
tracks_df

,id,name,popularity,duration_ms,explicit,id_artist,release_date,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0RNxWy0PC3AyH4ThH3aGK6,None,NaN,201467,0,19eLuQmk9aCobbVDHc6eek,1929,0.673,0.3770,0,-14.141,0.0697,0.5860,0.000000,0.3320,0.7130,88.973
1,2W889aLIKxULEefrleFBFI,Someone to Watch Over Me,54.0,198000,0,1Mxqyy3pSjf8kZZL4QVxS0,1943,0.204,0.1510,2,-17.842,0.0418,0.9470,0.000009,0.3210,0.1340,91.783
2,4Pnzw1nLOpDNV6MKI5ueIR,None,55.0,199000,0,1Mxqyy3pSjf8kZZL4QVxS0,1944,0.295,0.0826,1,-19.569,0.0367,0.9840,0.000358,0.1560,0.1690,128.600
3,7GLmfKOe5BfOXk7334DoKt,Saturday Night (Is The Loneliest Night In The ...,NaN,163000,0,1Mxqyy3pSjf8kZZL4QVxS0,1944,0.561,0.3350,9,-11.093,0.0499,0.8400,0.000002,0.7880,0.5900,126.974
4,6kD1SNGPkfX9LwaGd1FG92,Put Your Dreams Away (For Another Day),NaN,186173,0,1Mxqyy3pSjf8kZZL4QVxS0,1944,0.197,0.0546,1,-22.411,0.0346,0.9500,0.276000,0.1520,0.1000,90.150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22407,0LcNMuOiULmxJK3bdHTfDF,Missing You,65.0,214987,0,7gAppWoH7pcYmphCVTXkzs,2019-04-19,0.537,0.6470,2,-7.370,0.0519,0.1680,0.000000,0.1080,0.5310,79.770
22408,1uviKYHZuM4uINK33F7sCt,Fix It to Break It,70.0,198799,0,7okSU80WTrn4LXlyXYbX3P,2020-03-27,0.493,0.4610,2,-8.524,0.0456,0.8450,0.000000,0.1150,0.3500,51.414
22409,None,Lotus Inn,63.0,195868,0,2jnIB6XdLvnJUeNTy5A0J2,2020-12-04,0.578,0.7400,2,-5.080,0.0721,0.0119,0.000000,0.1800,0.4950,155.984
22410,27kcZEJvhkb1rzZS9gCpdA,None,67.0,202355,0,7okSU80WTrn4LXlyXYbX3P,2020-11-27,0.590,0.4430,0,-5.662,0.0722,0.6180,0.000000,0.1100,0.3420,149.447


### 2. Analiza danych

#### Zbiór artystów

Pierwszą rzeczą w analizie zbioru artystów jest sprawdzenie rozmiaru posiadanych danych, informacji o kolumnach za pomocą funkcji shape i info.

In [326]:
artists_df.shape

(1667, 3)

In [327]:
artists_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1667 entries, 0 to 1666
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      1667 non-null   object
 1   name    1667 non-null   object
 2   genres  1339 non-null   object
dtypes: object(3)
memory usage: 39.2+ KB


Użyto także funkcji `describe`, która pozwoli nam użyć metryk celem podsumowania kolumn występujących w DataFramie. Sprawdzono także występujące wartości typu None w danych.

In [328]:
artists_df.describe(include=object)

,id,name,genres
count,1667,1667,1339
unique,1347,1667,1118
top,-1,Lil Nas X,"[k-pop, k-pop boy group]"
freq,321,1,11


In [329]:
artists_df.isnull().sum()

id          0
name        0
genres    328
dtype: int64

Można zauważyć kilka rzeczy na podstawie `describe` i podsumowania wartości None:
1. Unikalnych wartości dla kolumny 'id' jest 1347, podczas gdy wszystkich wierszy (oraz wartości dla kolumny 'id') jest 1667. Oznacza to, że prawdopodobnie mamy do czynienia z anomalią/błędem w danych.
2. Unikalnych wartości dla kolumny 'genres' jest 1118, a wszystkich wartości 1339, podczas gdy rozmiar DataFrame to 1667 x 3. Oznacza to zarówno możliwe błędy w danych lub inną formę reprezentacji danych niż standardowo jedna wartość w jednej komórce. Na podstawie funkcji `isnull()` wnioskujemy o 328 wartościach typu None w kolumnie 'genres', co łącznie daje nam 1667 wartości.
3. Unikalnych wartości dla kolumny 'name' jest 1667 - prawdopodobnie nie mamy tutaj błędów w danych, możliwe problemy z nazwami/imionami artystów w nietypowym systemie kodowania (np.: nazwy artystów w języku chińskim).

Sprawdzimy teraz każdą obserwację.
Zajmijmy się kolumną 'id'. Sprawdzimy w jakiej formie są dane w tej kolumnie oraz sprawdzimy ewentualne błędy.


In [330]:
artists_df['id']

0       7jVv8c5Fj3E9VhNjxT4snq
1       0xRXCcSX89eobfrshSVdyu
2                           -1
3       4f7KfxeHq9BiylGmyXepGt
4                           -1
                 ...          
1662    6AMd49uBDJfhf30Ak2QR5s
1663    6qgnBH6iDM91ipVXv28OMu
1664    5q8HGNo0BjLWaTAhRtbwxa
1665                        -1
1666    46MWeeHNVMYRIIofQBEX98
Name: id, Length: 1667, dtype: object

In [331]:
artists_df[artists_df['id'].duplicated()]['id'].value_counts()

-1    320
Name: id, dtype: int64

In [332]:
artists_df["id"].value_counts().head()

-1                        321
7jVv8c5Fj3E9VhNjxT4snq      1
1moxjboGR7GNWYIMWsRjgG      1
49tQo2QULno7gxHutgccqF      1
37M5pPGs6V1fchFJSgCguX      1
Name: id, dtype: int64

Pierwsze sprawdzenie już daje nam możliwą przyczynę, błąd danych w kolumnie 'id'. Policzyliśmy występujące duplikaty w wartościach i wyświetliliśmy je. Mamy 320 wartości "-1" w kolumnie 'id'. Stąd wcześniej widzieliśmy jedynie 1347 unikalnych wartości w kolumnie 'id'. Łącznie daję nam 1667 wartości, tyle ile mieliśmy wierszy w DataFrame.

Z punktu widzenia modelowania, jest to istotny błąd w danych. Nie wiemy wtedy, jakie piosenki należą do danego artysty w zbiorze danych tracks.

Zajmijmy się teraz drugą obserwacją i sprawdźmy kolumnę z gatunkami artystów.

In [333]:
artists_df[artists_df['genres'].isna()]

,id,name,genres
6,25UNJbwGZSQKvz5cPLWlv3,Los Dos Carnales,None
10,0n6sKrG0xKAf8xmdqeNGke,Los Legendarios,None
11,0IxgA9wO4Op3CSnPlhfwV1,Kevin,None
18,0vLsqW05dyLvjuKKftAEGA,Die drei ??? Kids,None
29,-1,CORPSE,None
...,...,...,...
1627,3YQKmKGau1PzlVlkL1iodx,Twenty One Pilots,None
1631,4oUHIQIBe0LHzYfvXNW4QM,Morgan Wallen,None
1636,3Y2MLcrD3ZQXRXmtn4fZQx,SpotemGottem,None
1643,55fhWPvDiMpLnE4ZzNXZyW,Noah Cyrus,None


Mamy 328 wartości None, które są poważnymi błędami w danych. Nie jesteśmy wtedy w stanie użyć takich danych do dalszej analizy związanej z piosenkami oraz artystami, nie mając gatunków przypisanych do artystów.
Dla dalszej analizy danych związanych z gatunkami, usunięto 328 rekordów, które są typu None. 

In [334]:
artists_df_genres = artists_df[~artists_df['genres'].isna()]

In [335]:
artists_df_genres.reset_index(inplace=True)

In [336]:
artists_df_genres.shape

(1339, 4)

In [337]:
artists_df_genres.describe()

,index
count,1339.000000
mean,833.455564
std,483.064750
min,0.000000
25%,415.000000
50%,830.000000
75%,1257.500000
max,1666.000000


In [338]:
# sprawdzenie czy listy w kolumnie 'genres' są listami, a nie np.: stringami
for i, l in enumerate(artists_df_genres["genres"]):
    if not isinstance(l, list):
        print("list",i,"is",type(l))

In [339]:
def check_list(_list):
    for element in _list:
        if not isinstance(element, str):
            return True
        else:
            return False

In [340]:
artists_df_genres[artists_df_genres['genres'].apply(check_list)]

,index,id,name,genres


Mamy do czynienia z kolumną, w której wartości są listami, w których mamy elementy typu string. Wiemy, że usunęliśmy 328 rekordów związanych z wartością None w kolumnie 'genres'. Jest to jednak bezpośrednie sprawdzenie, że listy z gatunkami nie ma w danej komórce, stąd dodatkowo sprawdzamy, czy każdy element w liście jest typu string.
Sprawdźmy teraz, jakie gatunki mamy w 1339 wierszach, które nam pozostały.

In [341]:
def flatten_series(series):
    return pd.Series([x for _list in series for x in _list])

In [342]:
flatten_genres = flatten_series(artists_df_genres['genres'])

In [343]:
flatten_genres_size = flatten_genres.size
flatten_genres_size

5545

In [344]:
flatten_genres.value_counts()

pop                 281
dance pop           206
pop rap             141
rap                 132
post-teen pop       128
                   ... 
uk americana          1
modern folk rock      1
merseybeat            1
panamanian pop        1
deep ccm              1
Length: 774, dtype: int64

In [345]:
flatten_genres.value_counts().head(10)

pop              281
dance pop        206
pop rap          141
rap              132
post-teen pop    128
rock             111
latin            105
hip hop          103
trap              83
trap latino       83
dtype: int64

In [346]:
sum(flatten_genres.value_counts() == 1)

331

In [347]:
sum(flatten_genres.value_counts() == 2)

132

In [348]:
sum(flatten_genres.value_counts() == 3)

71

In [349]:
sum(flatten_genres.value_counts() == 4)

33

In [350]:
temp = flatten_genres.value_counts() == 1

In [351]:
temp[temp].index

Index(['chicano rap', 'destroy techno', 'australian psych', 'future garage',
       'deep underground hip hop', 'japanese soul', 'moombahton',
       'celtic rock', 'south african house', 'bow pop',
       ...
       'new age piano', 'reading', 'future house', 'german romanticism',
       'kansas city hip hop', 'uk americana', 'modern folk rock', 'merseybeat',
       'panamanian pop', 'deep ccm'],
      dtype='object', length=331)

In [352]:
flatten_genres.isin(['pop dance']).value_counts()

False    5472
True       73
dtype: int64

In [353]:
flatten_genres.isin(['dance pop']).value_counts()

False    5339
True      206
dtype: int64

Możemy zauważyć, że w gatunkach mamy bardzo mało gatunków, które są jednoznacznie określone. Oznacza to, że istnieje wiele podgatunków, które zaliczają się do jednego gatunku. Na podstawie obecnych danych, trzeba pogrupować je w jednolite gatunki, tj. przykładowo pop, dance pop, pop rap powinny tworzyć jeden gatunek pop - bez takiego przypisywania podgatunków do gatunków mamy bardzo mało informacji o podgatunkach występujących w liczbie np.: 1, 2, 3, 4 (jest ich łącznie 567 w porównaniu do 5545 wszystkich gatunków).

Widzimy jednak, że mamy przeważającą liczbę podgatunków związanych z gatunkiem pop, 4 na 5 najczęściej występujących. Jest to w tym przypadku klasa większościowa, która może negatywnie wpłynąć na modelowanie.

 Problemem są podgatunki takie jak np.: pop rap, ponieważ przy użyciu heurystyk: przykładowo sprawdzania czy w gatunku istnieje string "pop" lub "rap" i na tej podstawie dodawania podgatunku do gatunku, mielibyśmy sytuację, w której podgatunek pop rap należy zarówno do gatunku pop jak i rap. Mamy także problem w przypadku bezpośrednich heurystyk z podgatunkami, które w rzeczywistości oznaczają jeden podgatunek, jak np.: "dance pop" jest takim samym podgatunkiem jak "pop dance".

 Jedną z możliwości jest zastosowanie pierwszego modelu związanego z metodami asocjacyjnymi. Dzięki nim, będziemy w stanie zredukować liczbę występujących podgatunków i wprowadzić kilkanaście głównych gatunków do kolejnego modelu.

 Nie zauważono błędów związanych z kodowaniem nazw w utf-8. Mamy ich 99 w kolumnie "name", ale są poprawnie wyświetlane.



#### Zbiór utworów

Pierwszą rzeczą w analizie zbiorze piosenek jest sprawdzenie rozmiaru posiadanych danych, informacji o kolumnach za pomocą funkcji shape i info.

In [354]:
tracks_df.shape

(22412, 17)

In [355]:
tracks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22412 entries, 0 to 22411
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                17932 non-null  object 
 1   name              17869 non-null  object 
 2   popularity        17976 non-null  float64
 3   duration_ms       22412 non-null  int64  
 4   explicit          22412 non-null  int64  
 5   id_artist         17945 non-null  object 
 6   release_date      22412 non-null  object 
 7   danceability      22412 non-null  float64
 8   energy            22412 non-null  float64
 9   key               22412 non-null  int64  
 10  loudness          22412 non-null  float64
 11  speechiness       22412 non-null  float64
 12  acousticness      22412 non-null  float64
 13  instrumentalness  22412 non-null  float64
 14  liveness          22412 non-null  float64
 15  valence           22412 non-null  float64
 16  tempo             22412 non-null  float6

In [356]:
tracks_df.describe()

,popularity,duration_ms,explicit,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
count,17976.000000,2.241200e+04,22412.000000,22412.000000,22412.000000,22412.000000,22412.000000,22412.000000,22412.000000,22412.000000,22412.000000,22412.000000,22412.000000
mean,61.403204,2.301383e+05,0.166607,0.599096,0.648017,5.278824,-7.196227,0.082914,0.267758,0.040220,0.193141,0.518000,121.911945
std,8.063571,7.209562e+04,0.372633,0.158259,0.209577,3.558813,3.738098,0.089317,0.276111,0.152828,0.167943,0.244477,29.615254
min,51.000000,3.062200e+04,0.000000,0.000000,0.000103,0.000000,-44.410000,0.000000,0.000001,0.000000,0.012000,0.000000,0.000000
25%,55.000000,1.914930e+05,0.000000,0.495000,0.508000,2.000000,-8.614500,0.033900,0.029800,0.000000,0.094400,0.325000,97.988000
50%,60.000000,2.206670e+05,0.000000,0.608000,0.672000,5.000000,-6.393000,0.046300,0.165000,0.000003,0.125000,0.512000,120.041000
75%,67.000000,2.562400e+05,0.000000,0.715000,0.816000,8.000000,-4.850000,0.086000,0.455000,0.000523,0.240000,0.713000,140.078000
max,99.000000,4.120258e+06,1.000000,0.980000,0.999000,11.000000,0.642000,0.944000,0.996000,0.998000,0.997000,0.991000,220.099000


In [357]:
tracks_df.describe(include=object)

,id,name,id_artist,release_date
count,17932,17869,17945,22412
unique,17932,14689,1392,3907
top,0RNxWy0PC3AyH4ThH3aGK6,Animals,06HL4z0CvFAxyc27GXpf02,2013-01-01
freq,1,10,255,268


In [358]:
tracks_df.isnull().sum()

id                  4480
name                4543
popularity          4436
duration_ms            0
explicit               0
id_artist           4467
release_date           0
danceability           0
energy                 0
key                    0
loudness               0
speechiness            0
acousticness           0
instrumentalness       0
liveness               0
valence                0
tempo                  0
dtype: int64

Możemy zauważyć, że występują wartości typu None w czterech kolumnach: "id", "name", "popularity", "id_artist". Błędy typu id jesteśmy w stanie uzupełnić generując inne id np.: przypisując brakującym piosenkom kolejne cyfry, samo id piosenki nie jest nam szczególnie potrzebne w przypadku modelowania. Istotnymi błędami są wartości None w pozostałych trzech kolumnach. 

1. Kolumna "name": brak nazw dla piosenek nie jest możliwy do uzupełnienia, ponieważ możemy mieć rekordy o podobnych cechach takich jak energy, danceability, key, loudness itd. dla takich samych piosenek - brak możliwości wykrycia duplikatów piosenek.
2. Kolumna "id_artist": brak id artysty uniemożliwia nam mapowanie artystów i gatunków należących do nich do ich piosenek. Jest to istotny błąd, który uniemożliwia użycie tych danych do modelowania.
3. Kolumna "popularity": brak popularności dla danej piosenki może nie być kluczową cechą w przypadku modelowania związanego z przypisywaniem gatunków do autorów, jest to jednak brak, który ciężko wypełnić bez ponownego przesłania nowych danych.

Celem dalszej analizy cech, spróbujmy usunąć braki związane z trzema wymienionymi kolumnami. Czwartą kolumnę, "id" sprawdzimy po usunięciu braków w trzech kolumnach.

In [359]:
tracks_df_clean = tracks_df[~tracks_df['name'].isna()]
tracks_df_clean = tracks_df_clean[~tracks_df_clean['id_artist'].isna()]
tracks_df_clean = tracks_df_clean[~tracks_df_clean['popularity'].isna()]

In [360]:
tracks_df_clean.isnull().sum()

id                  2303
name                   0
popularity             0
duration_ms            0
explicit               0
id_artist              0
release_date           0
danceability           0
energy                 0
key                    0
loudness               0
speechiness            0
acousticness           0
instrumentalness       0
liveness               0
valence                0
tempo                  0
dtype: int64

In [361]:
tracks_df_clean.reset_index(inplace=True) # reset indexu celem poprawnego zliczania danych
tracks_df_clean.pop('index')

0            1
1            6
2            8
3            9
4           10
         ...  
11446    22402
11447    22405
11448    22407
11449    22408
11450    22409
Name: index, Length: 11451, dtype: int64

In [362]:
tracks_df_clean

,id,name,popularity,duration_ms,explicit,id_artist,release_date,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,2W889aLIKxULEefrleFBFI,Someone to Watch Over Me,54.0,198000,0,1Mxqyy3pSjf8kZZL4QVxS0,1943,0.204,0.1510,2,-17.842,0.0418,0.9470,0.000009,0.3210,0.1340,91.783
1,5DIVWgTeJ2fPIxaY9e7ZKn,I've Got A Crush On You,56.0,196440,0,5V0MlUE1Bft0mbLlND7FJz,1950-09-11,0.455,0.1060,2,-14.086,0.0422,0.9860,0.000000,0.1220,0.2580,66.807
2,0Hsc0sIaxOxXBZbT3ms2oj,In The Wee Small Hours Of The Morning - Remast...,60.0,180933,0,1Mxqyy3pSjf8kZZL4QVxS0,1955,0.290,0.0874,0,-16.119,0.0346,0.8560,0.000957,0.1090,0.0734,114.997
3,3aEJMh1cXKEjgh52claxQp,I've Got You Under My Skin - Remastered 1998,68.0,223760,0,1Mxqyy3pSjf8kZZL4QVxS0,1956-03,0.585,0.2470,1,-12.612,0.0400,0.4520,0.000009,0.1070,0.5910,127.150
4,None,You Make Me Feel So Young - Remastered 1998,66.0,177400,0,1Mxqyy3pSjf8kZZL4QVxS0,1956-03,0.577,0.3060,9,-11.559,0.0427,0.3830,0.000000,0.1800,0.5680,125.166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11446,6rsm9NTgl9kKPatf7S1yCS,Talk,61.0,190347,0,2jnIB6XdLvnJUeNTy5A0J2,2018-08-31,0.795,0.7000,6,-5.221,0.1070,0.0588,0.000000,0.0977,0.7980,120.002
11447,None,Soothing Dog Music,56.0,45733,0,1Cj9j44XMhQeNHAwWJRCKd,2018-12-24,0.278,0.0614,5,-20.268,0.0474,0.9930,0.961000,0.1240,0.1320,75.139
11448,0LcNMuOiULmxJK3bdHTfDF,Missing You,65.0,214987,0,7gAppWoH7pcYmphCVTXkzs,2019-04-19,0.537,0.6470,2,-7.370,0.0519,0.1680,0.000000,0.1080,0.5310,79.770
11449,1uviKYHZuM4uINK33F7sCt,Fix It to Break It,70.0,198799,0,7okSU80WTrn4LXlyXYbX3P,2020-03-27,0.493,0.4610,2,-8.524,0.0456,0.8450,0.000000,0.1150,0.3500,51.414


In [363]:
tracks_df_clean_rows_size = tracks_df_clean.shape[0]
tracks_df_clean_rows_size

11451

In [364]:
tracks_df_clean_rows = np.arange(tracks_df_clean_rows_size)
tracks_df_clean_rows = pd.Series(tracks_df_clean_rows)

In [365]:
tracks_df_clean['id'].fillna(tracks_df_clean_rows, inplace=True)

In [366]:
tracks_df_clean.isnull().sum()

id                  0
name                0
popularity          0
duration_ms         0
explicit            0
id_artist           0
release_date        0
danceability        0
energy              0
key                 0
loudness            0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
dtype: int64

Uzupełniono brakujące 2303 id kolejnymi cyframi od 0 do 11450. Nastąpiła znaczna redukcja danych, z 22412 do 11451. Sprawdźmy także, ile artystów ze zbioru artystów matchuje się w piosenkach (po polu id_artist).

In [367]:
(artists_df['id'] == -1).value_counts()

False    1346
True      321
Name: id, dtype: int64

Mamy jednak 321 artystów bez id, które celem dalszej analizy musimy usunąć, ponieważ nie znajdziemy dla nich matchy w zbiorze piosenek.

In [368]:
artists_df = artists_df[~(artists_df['id'] == -1)]
artists_df.reset_index(inplace=True)


In [369]:
artists_df.drop('index', inplace=True, axis=1)

C:\Users\rwole\AppData\Local\Temp\ipykernel_14016\683162473.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  artists_df.drop('index', inplace=True, axis=1)


In [370]:
artists_df

,id,name,genres
0,7jVv8c5Fj3E9VhNjxT4snq,Lil Nas X,"[country rap, lgbtq+ hip hop, pop rap, queer c..."
1,0xRXCcSX89eobfrshSVdyu,MEDUZA,"[dance pop, edm, pop dance, pop house, tropica..."
2,4f7KfxeHq9BiylGmyXepGt,Tanishk Bagchi,"[desi pop, filmi, modern bollywood]"
3,523y9KSneKh6APd1hKxLuF,Master KG,[south african house]
4,25UNJbwGZSQKvz5cPLWlv3,Los Dos Carnales,None
...,...,...,...
1341,3b8QkneNDz4JHKKKlLgYZg,Florida Georgia Line,"[contemporary country, country, country pop, c..."
1342,6AMd49uBDJfhf30Ak2QR5s,Coi Leray,"[pop r&b, r&b, trap queen, viral rap]"
1343,6qgnBH6iDM91ipVXv28OMu,KAYTRANADA,"[escape room, indie soul, lgbtq+ hip hop]"
1344,5q8HGNo0BjLWaTAhRtbwxa,Brantley Gilbert,"[contemporary country, country, country road, ..."


In [371]:
artists_ids = artists_df['id'].to_list()
artists_ids[:10]

['7jVv8c5Fj3E9VhNjxT4snq',
 '0xRXCcSX89eobfrshSVdyu',
 '4f7KfxeHq9BiylGmyXepGt',
 '523y9KSneKh6APd1hKxLuF',
 '25UNJbwGZSQKvz5cPLWlv3',
 '6vTqEFbTtTRJsuIpzZgjxi',
 '44mEtidu0VdRkIqO4IbkNa',
 '0n6sKrG0xKAf8xmdqeNGke',
 '0IxgA9wO4Op3CSnPlhfwV1',
 '4rTv3Ejc7hKMtmoBOK1B4T']

In [372]:
(tracks_df_clean['id_artist'].isin(artists_ids)).value_counts()

True     9092
False    2359
Name: id_artist, dtype: int64

Możemy zauważyć, że mamy 9092 piosenek, u których jesteśmy w stanie stwierdzić, w jakim mniej więcej gatunku tworzy artysta i na tej podstawie wnioskować o gatunku piosenki. Jeśli w ramach naszego pierwszego modelowania mielibyśmy określone gatunki z wliczonymi podgatunkami, to możemy użyć 9092 z tych danych, aby określić gatunek artysty na podstawie jego piosenki. 2359 danych nie jest użyteczne w kontekście naszego założonego modelowania, ponieważ będziemy mieć sytuację, w której piosenka nie ma przypisanego autora z naszego zbioru autorów, a co za tym idzie, nie będziemy mieli informacji o gatunkach przypisanych do autora.